# Experiment 3: Deep PDE Solving with cPIKAN

## Allen-Cahn Equation

In [ ]:
import jax
import jax.numpy as jnp

jax.config.update("jax_default_matmul_precision", "highest")

from src.equations import ac_res as pde_res

from src.utils import _get_adam, _get_pde_collocs, _get_ic_collocs, model_eval, count_params, _get_colloc_indices, grad_norm
from src.kan import KAN

import numpy as np

import optax
from flax import nnx

import os

# Create the directory if it doesn't exist
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)

# Define the experiment
experiment_name = "comparative_pde"
results_file = os.path.join(results_dir, f"{experiment_name}.csv")

# Define the file header
header = "pde, init, width, depth, run, l2"

# Check if the file exists and write the header if it doesn't
if not os.path.exists(results_file):
    with open(results_file, "w") as file:
        file.write(header + "\n")
        
seed = 42

### Basic Functions

In [ ]:
# PDE Loss
def pde_loss(model, l_E, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Get new RBA weights
    abs_res = jnp.abs(residuals)
    l_E_new = (RBA_gamma*l_E) + (RBA_eta*abs_res/jnp.max(abs_res)) # shape (batch_size, 1)

    # Multiply by RBA weights
    w_resids = l_E_new * residuals # shape (batch_size, 1)

    # Reshape residuals for causal training
    residuals = w_resids.reshape(num_chunks, -1) # shape (num_chunks, points)

    # Get average loss per chunk
    loss = jnp.mean(residuals**2, axis=1)

    # Get causal weights
    weights = jax.lax.stop_gradient(jnp.exp(-causal_tol * (M @ loss)))

    # Weighted loss
    weighted_loss = jnp.mean(weights * loss)

    return weighted_loss, l_E_new


# IC Loss
def ic_loss(model, l_I, ic_collocs, ic_data):

    # Residual
    ic_res = model(ic_collocs) - ic_data

    # Get new RBA weights
    abs_res = jnp.abs(ic_res)
    l_I_new = (RBA_gamma*l_I) + (RBA_eta*abs_res/jnp.max(abs_res))

    # Multiply by RBA weights
    w_resids = l_I_new * ic_res

    # Loss
    loss = jnp.mean(w_resids**2)

    return loss, l_I_new


@nnx.jit
def train_step(model, optimizer, collocs, ic_collocs, ic_data, λ_E, λ_I, l_E, l_I):

    # PDE loss
    (loss_E, l_E_new), grads_E = nnx.value_and_grad(pde_loss, has_aux=True)(model, l_E, collocs)

    # IC loss
    (loss_I, l_I_new), grads_I = nnx.value_and_grad(ic_loss, has_aux=True)(model, l_I, ic_collocs, ic_data)
    
    # Compute total loss
    loss = λ_E*loss_E + λ_I*loss_I

    # Compute total gradients
    grads = jax.tree_util.tree_map(lambda g1, g2: λ_E * g1 + λ_I * g2, grads_E, grads_I)

    # Optimizer step
    optimizer.update(grads)

    return loss, grads_E, grads_I, l_E_new, l_I_new


@nnx.jit
def get_RAD_indices(model, collocs_pool, old_indices, l_E, l_E_pool):

    # Apply updates from old indices to pool
    updated_pool = l_E_pool.at[old_indices].set(l_E)

    # Get full residuals
    resids = pde_res(model, collocs_pool)
    
    # Multiply by RBA weights
    w_resids = updated_pool * resids
    
    # Get absolute
    wa_resids = jnp.abs(w_resids)

    # Raise to power rad_a
    ea = jnp.power(wa_resids, rad_a)
    
    # Divide by mean and add rad_c
    px = (ea/jnp.mean(ea)) + rad_c
    
    # Normalize
    px_norm = (px / jnp.sum(px))[:,0]

    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=px_norm, seed=seed)

    return sorted_indices, updated_pool

### Data & Grid-Search Parameters

In [ ]:
# Collocation points and ICs
collocs_pool = _get_pde_collocs(ranges = [(0,1), (-1,1)], sample_size = 400)
ic_collocs = _get_ic_collocs(x_range = (-1, 1), sample_size = 2**6)

# Custom for each PDE
ic_data = ((ic_collocs[:,1]**2)*jnp.cos(jnp.pi*ic_collocs[:,1])).reshape(-1,1)

# Reference solution
ref = np.load('data/allen_cahn.npz')
refsol = jnp.array(ref['usol'])

N_t, N_x = ref['usol'].shape
t, x = ref['t'].flatten(), ref['x'].flatten()
T, X = jnp.meshgrid(t, x, indexing='ij')
coords = jnp.hstack([T.flatten()[:, None], X.flatten()[:, None]])

In [ ]:
# Training epochs
num_epochs = 100_000

# Scheduler configurations
learning_rate = 1e-3
decay_steps = 2000
decay_rate = 0.9
warmup_steps = 1000

# Define causal training parameters
causal_tol = 1.0
num_chunks = 32
M = jnp.triu(jnp.ones((num_chunks, num_chunks)), k=1).T

# Define Grad Norm parameters
grad_mixing = 0.9
f_grad_norm = 1000

# Define resampling parameters
batch_size = 2**12
f_resample = 2000
rad_a = 1.0
rad_c = 1.0

# Define RBA parameters
RBA_gamma = 0.999
RBA_eta = 0.01

# Define model parameters
n_in = collocs_pool.shape[1]
n_out = 1
D = 5
period_axes = {1: jnp.pi}
rff_std = None

glorot_init = {'type': 'glorot', 'gain': None, 'norm_pow': 0, 'distribution': 'uniform', 'sample_size': 10000}
default_init = {'type': 'default'}

In [ ]:
# Architecture settings
widths = [8, 16, 32]
depths = [2, 4, 6, 8, 10, 12]

### Grid Search

In [ ]:
# Procedure
for init_scheme in [default_init, glorot_init]:

    scheme_title = "glorot" if init_scheme == glorot_init else "default"

    # Grid search over depth size
    for num_layers in depths:

        # Grid search over width size
        for n_hidden in widths:
            
            print(f"Training model with depth = {num_layers} and width = {n_hidden} (Initilization = {scheme_title}).")

            for run in [3, 5, 6, 7, 13]:

                # Initialize RBA weights - full pool
                l_E_pool = jnp.ones((collocs_pool.shape[0], 1))
                # Also get RBAs for ICs
                l_I = jnp.ones((ic_collocs.shape[0], 1))
            
                # Get starting collocation points & RBA weights
                sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
                
                collocs = collocs_pool[sorted_indices]
                l_E = l_E_pool[sorted_indices]
                
                # Get opt_type
                opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

                # Define model
                model = KAN(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_layers = num_layers, D = D,
                            init_scheme = init_scheme, period_axes = period_axes, rff_std = rff_std,
                            seed = seed+run)

                if run == 1:
                    print(f"Initialized model with {count_params(model)} parameters.")
                
                # Define global loss weights
                λ_E = jnp.array(1.0, dtype=float)
                λ_I = jnp.array(1.0, dtype=float)

                # Set optimizer
                optimizer = nnx.Optimizer(model, opt_type)
            
                # Start training
                for epoch in range(num_epochs):
                
                    loss, grads_E, grads_I, l_E, l_I = train_step(model, optimizer, collocs, ic_collocs, ic_data, λ_E, λ_I, l_E, l_I)
                
                    # Perform grad norm
                    if (epoch != 0) and (epoch % f_grad_norm == 0):
                
                        λ_Ε, λ_I = grad_norm(grads_E, grads_I, λ_E, λ_I, grad_mixing)
                
                    # Perform RAD
                    if (epoch != 0) and (epoch % f_resample == 0):
            
                        # Get new indices after resampling
                        sorted_indices, l_E_pool = get_RAD_indices(model, collocs_pool, sorted_indices, l_E, l_E_pool)
                        # Set new batch of collocs and l_E
                        collocs = collocs_pool[sorted_indices]
                        l_E = l_E_pool[sorted_indices]
            
                # Evaluate
                output = model(coords).reshape(refsol.shape)
                l2error = jnp.linalg.norm(output-refsol)/jnp.linalg.norm(refsol)
            
                # Log results
                new_row = f"ac, {scheme_title}, {n_hidden}, {num_layers}, {run}, {l2error}"
                                
                # Append the row to the file
                with open(results_file, "a") as rfile:
                    rfile.write(new_row + "\n")

                print(f"\t{run}. Final Rel. L2 Error: {l2error:.2e}")

## Burgers' Equation

In [ ]:
from src.equations import burgers_res as pde_res

from src.wrappers import BurgersKAN

### Basic Functions

In [ ]:
# PDE Loss
def pde_loss(model, l_E, collocs):

    residuals = pde_res(model, collocs) # shape (batch_size, 1)

    # Get new RBA weights
    abs_res = jnp.abs(residuals)
    l_E_new = (RBA_gamma*l_E) + (RBA_eta*abs_res/jnp.max(abs_res)) # shape (batch_size, 1)

    # Multiply by RBA weights
    w_resids = l_E_new * residuals # shape (batch_size, 1)

    # Reshape residuals for causal training
    residuals = w_resids.reshape(num_chunks, -1) # shape (num_chunks, points)

    # Get average loss per chunk
    loss = jnp.mean(residuals**2, axis=1)

    # Get causal weights
    weights = jax.lax.stop_gradient(jnp.exp(-causal_tol * (M @ loss)))

    # Weighted loss
    weighted_loss = jnp.mean(weights * loss)

    return weighted_loss, l_E_new


# IC Loss
def ic_loss(model, l_I, ic_collocs, ic_data):

    # Residual
    ic_res = model(ic_collocs) - ic_data

    # Get new RBA weights
    abs_res = jnp.abs(ic_res)
    l_I_new = (RBA_gamma*l_I) + (RBA_eta*abs_res/jnp.max(abs_res))

    # Multiply by RBA weights
    w_resids = l_I_new * ic_res

    # Loss
    loss = jnp.mean(w_resids**2)

    return loss, l_I_new


@nnx.jit
def train_step(model, optimizer, collocs, ic_collocs, ic_data, λ_E, λ_I, l_E, l_I):

    # PDE loss
    (loss_E, l_E_new), grads_E = nnx.value_and_grad(pde_loss, has_aux=True)(model, l_E, collocs)

    # IC loss
    (loss_I, l_I_new), grads_I = nnx.value_and_grad(ic_loss, has_aux=True)(model, l_I, ic_collocs, ic_data)
    
    # Compute total loss
    loss = λ_E*loss_E + λ_I*loss_I

    # Compute total gradients
    grads = jax.tree_util.tree_map(lambda g1, g2: λ_E * g1 + λ_I * g2, grads_E, grads_I)

    # Optimizer step
    optimizer.update(grads)

    return loss, grads_E, grads_I, l_E_new, l_I_new


@nnx.jit
def get_RAD_indices(model, collocs_pool, old_indices, l_E, l_E_pool):

    # Apply updates from old indices to pool
    updated_pool = l_E_pool.at[old_indices].set(l_E)

    # Get full residuals
    resids = pde_res(model, collocs_pool)
    
    # Multiply by RBA weights
    w_resids = updated_pool * resids
    
    # Get absolute
    wa_resids = jnp.abs(w_resids)

    # Raise to power rad_a
    ea = jnp.power(wa_resids, rad_a)
    
    # Divide by mean and add rad_c
    px = (ea/jnp.mean(ea)) + rad_c
    
    # Normalize
    px_norm = (px / jnp.sum(px))[:,0]

    sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=px_norm, seed=seed)

    return sorted_indices, updated_pool

### Data & Grid-Search Parameters

In [ ]:
# Collocation points and ICs
collocs_pool = _get_pde_collocs(ranges = [(0,1), (-1,1)], sample_size = 400)
ic_collocs = _get_ic_collocs(x_range = (-1, 1), sample_size = 2**6)

# Custom for each PDE
ic_data = -jnp.sin(jnp.pi*ic_collocs[:,1]).reshape(-1,1)

# Reference solution
ref = np.load('data/burgers.npz')
refsol = jnp.array(ref['usol'])

N_t, N_x = ref['usol'].shape
t, x = ref['t'].flatten(), ref['x'].flatten()
T, X = jnp.meshgrid(t, x, indexing='ij')
coords = jnp.hstack([T.flatten()[:, None], X.flatten()[:, None]])

In [ ]:
# Training epochs
num_epochs = 100_000

# Scheduler configurations
learning_rate = 1e-3
decay_steps = 2000
decay_rate = 0.9
warmup_steps = 1000

# Define causal training parameters
causal_tol = 1.0
num_chunks = 32
M = jnp.triu(jnp.ones((num_chunks, num_chunks)), k=1).T

# Define Grad Norm parameters
grad_mixing = 0.9
f_grad_norm = 1000

# Define resampling parameters
batch_size = 2**12
f_resample = 2000
rad_a = 1.0
rad_c = 1.0

# Define RBA parameters
RBA_gamma = 0.999
RBA_eta = 0.01

# Define model parameters
n_in = collocs_pool.shape[1]
n_out = 1
D = 5
period_axes = None
rff_std = None

glorot_init = {'type': 'glorot', 'gain': None, 'norm_pow': 0, 'distribution': 'uniform', 'sample_size': 10000}
default_init = {'type': 'default'}

In [ ]:
# Architecture settings
widths = [8, 16, 32]
depths = [2, 4, 6, 8, 10, 12]

### Grid Search

In [ ]:
# Procedure
for init_scheme in [default_init, glorot_init]:

    scheme_title = "glorot" if init_scheme == glorot_init else "default"

    # Grid search over depth size
    for num_layers in depths:

        # Grid search over width size
        for n_hidden in widths:
            
            print(f"Training model with depth = {num_layers} and width = {n_hidden} (Initilization = {scheme_title}).")

            for run in [3, 5, 6, 7, 13]:

                # Initialize RBA weights - full pool
                l_E_pool = jnp.ones((collocs_pool.shape[0], 1))
                # Also get RBAs for ICs
                l_I = jnp.ones((ic_collocs.shape[0], 1))
            
                # Get starting collocation points & RBA weights
                sorted_indices = _get_colloc_indices(collocs_pool=collocs_pool, batch_size=batch_size, px=None, seed=seed)
                
                collocs = collocs_pool[sorted_indices]
                l_E = l_E_pool[sorted_indices]
                
                # Get opt_type
                opt_type = _get_adam(learning_rate=learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, warmup_steps=warmup_steps)

                # Define model
                model = BurgersKAN(n_in = n_in, n_out = n_out, n_hidden = n_hidden, num_layers = num_layers, D = D,
                            init_scheme = init_scheme, period_axes = period_axes, rff_std = rff_std,
                            seed = seed+run)

                if run == 1:
                    print(f"Initialized model with {count_params(model)} parameters.")
                
                # Define global loss weights
                λ_E = jnp.array(1.0, dtype=float)
                λ_I = jnp.array(1.0, dtype=float)

                # Set optimizer
                optimizer = nnx.Optimizer(model, opt_type)
            
                # Start training
                for epoch in range(num_epochs):
                
                    loss, grads_E, grads_I, l_E, l_I = train_step(model, optimizer, collocs, ic_collocs, ic_data, λ_E, λ_I, l_E, l_I)
                
                    # Perform grad norm
                    if (epoch != 0) and (epoch % f_grad_norm == 0):
                
                        λ_Ε, λ_I = grad_norm(grads_E, grads_I, λ_E, λ_I, grad_mixing)
                
                    # Perform RAD
                    if (epoch != 0) and (epoch % f_resample == 0):
            
                        # Get new indices after resampling
                        sorted_indices, l_E_pool = get_RAD_indices(model, collocs_pool, sorted_indices, l_E, l_E_pool)
                        # Set new batch of collocs and l_E
                        collocs = collocs_pool[sorted_indices]
                        l_E = l_E_pool[sorted_indices]
            
                # Evaluate
                output = model(coords).reshape(refsol.shape)
                l2error = jnp.linalg.norm(output-refsol)/jnp.linalg.norm(refsol)
            
                # Log results
                new_row = f"burgers, {scheme_title}, {n_hidden}, {num_layers}, {run}, {l2error}"
                                
                # Append the row to the file
                with open(results_file, "a") as rfile:
                    rfile.write(new_row + "\n")

                print(f"\t{run}. Final Rel. L2 Error: {l2error:.2e}")

## Plots

In [ ]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore", category=pd.errors.ParserWarning)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatterMathtext, NullLocator, NullFormatter
import seaborn as sns

# ===== configurable knobs =====
TITLE_FS   = 16
LABEL_FS   = 16
TICK_FS    = 14
LEGEND_FS  = 16
LINE_W     = 2.0
ALPHA_FILL = 0.25
PDE_ORDER  = ["burgers", "ac"]      # row order (change if you like)
WIDTHS     = [8, 16, 32]            # column order
INIT_LABEL = {"glorot": "Proposed Initialization", "default": "Default Initialization"}
PDE_LABEL = {"ac": "Allen-Cahn", "burgers": "Burgers"}

# color choice (2 distinct tones from a seaborn colormap)
_cmap = sns.color_palette("Spectral", as_cmap=True)
cmap_points = np.linspace(0, 1, 12)

color_indices = [1, -2]
_colors = [_cmap(cmap_points[i]) for i in color_indices]

def _set_log_ticks(ax):
    ax.set_yscale("log")
    ax.yaxis.set_major_locator(LogLocator(base=10.0))
    ax.yaxis.set_major_formatter(LogFormatterMathtext(base=10.0))
    ax.yaxis.set_minor_locator(NullLocator())
    ax.yaxis.set_minor_formatter(NullFormatter())
    ax.tick_params(axis="both", which="both", labelsize=TICK_FS)

def _plot_mean_sem(ax, x, y_mean, y_sem, label, color):
    line, = ax.plot(x, y_mean, label=label, linewidth=LINE_W, color=color, marker="o")
    ax.fill_between(x, y_mean - y_sem, y_mean + y_sem, alpha=ALPHA_FILL, color=color, linewidth=0)
    return line

def _prep_stats(df):
    # expect columns: pde, init, width, depth, run, l2
    g = df.groupby(["pde", "init", "width", "depth"])["l2"].agg(["mean", "sem"]).reset_index()
    # ensure depth is sorted numerically
    g = g.sort_values(["pde", "width", "init", "depth"]).reset_index(drop=True)
    return g

def plot_pde_grid(df):

    stats = _prep_stats(df)

    # ensure presence/ordering
    pdes   = [p for p in PDE_ORDER if p in stats["pde"].unique()]
    widths = [w for w in WIDTHS    if w in stats["width"].unique()]
    inits  = [i for i in ["default", "glorot"] if i in stats["init"].unique()]

    nrows, ncols = len(pdes), len(widths)
    if nrows == 0 or ncols == 0 or not inits:
        raise ValueError("No matching pdes/widths/inits to plot after filtering.")

    fig, axes = plt.subplots(
        nrows, ncols, figsize=(4 * ncols, 2.5 * nrows),
        sharex=True, sharey=False, constrained_layout=True
    )

    fig.set_constrained_layout_pads(w_pad=0.1, h_pad=0.1, hspace=0.06, wspace=0.06)
    
    # normalize axes to 2D array
    if nrows == 1 and ncols == 1:
        axes = np.array([[axes]])
    elif nrows == 1:
        axes = axes[np.newaxis, :]
    elif ncols == 1:
        axes = axes[:, np.newaxis]

    legend_handles = None

    for r, pde in enumerate(pdes):
        for c, width in enumerate(widths):
            ax = axes[r, c]
            _set_log_ticks(ax)

            # depths available for this (pde, width); keep numeric ascending order
            depths_here = (
                stats[(stats.pde == pde) & (stats.width == width)]["depth"]
                .drop_duplicates().sort_values().to_numpy()
            )

            # plot each init
            init_lines = []
            for j, init in enumerate(inits):
                sub = stats[(stats.pde == pde) & (stats.width == width) & (stats.init == init)]
                if sub.empty:
                    continue
                # align to complete x (depth) set to avoid broken fills
                sub = sub.set_index("depth").reindex(depths_here)
                if sub["mean"].isna().all():
                    continue
                h = _plot_mean_sem(
                    ax, depths_here, sub["mean"].to_numpy(), sub["sem"].fillna(0).to_numpy(),
                    label=INIT_LABEL.get(init, init.title()), color=_colors[j % len(_colors)]
                )
                init_lines.append(h)

            # titles on top row, row labels on leftmost col
            if r == 0:
                ax.set_title(f"Width = {width}", fontsize=TITLE_FS)
            if c == 0:
                ax.set_ylabel(r"Relative $L^2$ Error", fontsize=LABEL_FS)

            # x label only on bottom row
            if r == nrows - 1:
                ax.set_xlabel("Depth", fontsize=LABEL_FS)

            # fix depth ticks
            ax.set_xticks([2, 4, 6, 8, 10, 12])
            ax.set_xticklabels([2, 4, 6, 8, 10, 12])

            ax.grid(True, which="both", linestyle="--", alpha=0.5)

            # left annotate PDE name per row (nice, unobtrusive)
            if c == ncols - 1:
                ax.annotate(
                    PDE_LABEL.get(pde, pde),
                    xy=(1.08, 0.5), xycoords="axes fraction",
                    va="center", ha="left", rotation=90, fontsize=LABEL_FS
                )

            if legend_handles is None and init_lines:
                legend_handles = init_lines

    if legend_handles:
        fig.legend(
            legend_handles, [h.get_label() for h in legend_handles],
            loc="lower center", ncol=len(legend_handles), frameon=False, fontsize=LEGEND_FS,
            bbox_to_anchor=(0.5, -0.12)
        )

    plt.savefig(f"{plots_dir}/compar_pdes.pdf", format="pdf", bbox_inches="tight")
    plt.show()
    

In [ ]:
df = pd.read_csv("results/comparative_pde.csv", sep=', ')
plot_pde_grid(df)